In [18]:
import requests
import time
import pandas as pd
import os
import re

In [ ]:
# 엑셀 파일 불러오기
df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/병원 풀데이터.xlsx")
pharmacy_df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/약국 풀데이터.xlsx")

In [17]:
# 사전 설정
app_key = "mqiZDweuvg5SSqYKKqqGy4Tu26DQy8jT2vkv8QDo"
address_col = 'dutyAddr'

In [16]:
# 시도명 매핑
SIDO_MAP = {
    "서울": "서울특별시",
    "부산": "부산광역시",
    "대구": "대구광역시",
    "인천": "인천광역시",
    "광주": "광주광역시",
    "대전": "대전광역시",
    "울산": "울산광역시",
    "세종": "세종특별시",
    "경기": "경기도",
    "강원": "강원도",
    "충북": "충청북도",
    "충남": "충청남도",
    "전북": "전라북도",
    "전남": "전라남도",
    "경북": "경상북도",
    "경남": "경상남도",
    "제주": "제주도",
    "강원특별자치도": "강원도",
    "제주특별자치도": "제주도",
    "전북특별자치도": "전라북도"
    }

In [ ]:
def convert_to_jibun(road_address, index=None):
    # ✅ 콤마 제거 처리 (주소에 콤마가 포함돼 있으면 앞부분만 사용)
    if "," in road_address:
        original = road_address
        road_address = road_address.split(",")[0].strip()

    url = f"https://apis.openapi.sk.com/tmap/geo/convertAddress"
    headers = {
        "Accept": 'application/json',
        "version": "1",
        "searchTypCd": "NtoO",
        "reqAdd": road_address,
        "reqMulti": "M",
        "appKey": app_key,
    }

    try:
        response = requests.get(url, params=headers)
        data = response.json()

        if "ConvertAdd" not in data:
            print(f"❌ convert_to_jibun(): {index}행 - 'ConvertAdd' 키 없음 (응답: {data})")
            return None

        addr = data["ConvertAdd"]

        # 시도명 정식화
        sido_short = addr.get("upperDistName", "")
        sido = SIDO_MAP.get(sido_short, sido_short)

        sigungu = addr.get("middleDistName", "")
        dong = addr.get("legalLowerDistName", "")
        ri = addr.get("legalDetailName", "")

        # 지번 주소 조합
        jibun = f"{sido} {sigungu} {dong or ri}".strip()
        return jibun if jibun else None

    except Exception as e:
        print(f"❌ convert_to_jibun(): {index}행 요청 실패 - {e}")
        return None

In [ ]:
# 법정동 목록 추출
law_df = pd.read_csv("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/국토교통부_전국 법정동_20240802.csv", encoding="cp949")  # 인코딩은 CSV에 따라 조정

In [48]:
법정동_목록 = set(law_df["읍면동명"].dropna().unique())
시도_목록 = set(law_df["시도명"].dropna().unique())
시군구_목록 = set(law_df["시군구명"].dropna().unique())

In [ ]:
# ---------- 주소 추출 함수 ----------
def handle_parenthesis_address(addr, index=None):
    try:
        if "(" not in addr or ")" not in addr:
            return convert_to_jibun(addr, index)

        start = addr.find("(") + 1
        end = addr.find(")")
        if start >= end:
            return convert_to_jibun(addr, index)

        inner = addr[start:end].strip()
        candidates = [s.strip() for s in inner.split(",")]
        emd = next((s for s in candidates if s in 법정동_목록), None)

        # ❗emd 추출 실패한 경우만 디버깅 출력
        if emd is None:
            print(f"⚠️ [디버그] {index}행 - emd 추출 실패 | inner: '{inner}' | 후보군: {candidates}")

        if emd:
            parts = addr.split()
            if len(parts) < 2:
                return None

            sido_raw = parts[0]
            sgg = parts[1]
            sido = SIDO_MAP.get(sido_raw, sido_raw)

            if any(kw in sido for kw in ["특별시", "광역시", "세종"]):
                return f"{sido} {emd}"
            else:
                return f"{sido} {sgg} {emd}"
        else:
            new_addr = addr.split(",")[0].strip()
            return convert_to_jibun(new_addr, index)

    except Exception as e:
        print(f"❌ handle_parenthesis_address(): {index}행 오류 - {e}")
        return None

In [145]:
test_addr = "전북특별자치도 임실군 임실읍 봉황로 203 (다사랑의원)"
print(handle_parenthesis_address(test_addr))

⚠️ [디버그] None행 - emd 추출 실패 | inner: '다사랑의원' | 후보군: ['다사랑의원']
전라북도 임실군 임실읍


In [ ]:
# ---------- 3-2. 샘플 50건 중 괄호 있는 행은 직접 처리 ----------
# 1. 기존 df에서 앞 50행을 복사하여 새 데이터프레임 생성
sample_size = 50
df_sample = df.iloc[:sample_size].copy()

# 2. 지번주소 결과를 담을 리스트 생성
jibun_series = []

# 3. 주소 변환 및 저장 (괄호 있는 행은 직접 전처리)
count = 0

for i in range(sample_size):
    addr = df_sample[address_col].iloc[i]

    if "(" in addr and ")" in addr:
        jibun = handle_parenthesis_address(addr)
        jibun_series.append(jibun)
    else:
        jibun = convert_to_jibun(addr)
        if jibun and jibun.strip():
            jibun_series.append(jibun)
            count += 1
        else:
            jibun_series.append(None)
        time.sleep(0.2)

print(f"✅ 샘플 {sample_size}건 처리 완료 (API 성공: {count}건)")

# 4. "Jibun" 열을 dutyAddr 오른쪽에 삽입
insert_at = df_sample.columns.get_loc(address_col) + 1
df_sample.insert(insert_at, "Jibun", jibun_series)


✅ 샘플 50건 처리 완료 (API 성공: 8건)


In [ ]:
df_sample[['dutyAddr', 'Jibun']]

In [147]:
df['dutyAddr'].iloc[230]

'경상남도 양산시 동면 내송내안2길 29, 302, 303호'

In [151]:
handle_parenthesis_address('경상남도 양산시 동면 내송내안2길 29, 302, 303호')

🔧 [디버그] None행 - 콤마 이후 제거: '경상남도 양산시 동면 내송내안2길 29, 302, 303호' → '경상남도 양산시 동면 내송내안2길 29'


'경상남도 양산시 동면'

In [ ]:
# ---------- 3-3. 전체 데이터 처리 (1000개 단위 분할 처리) ----------
batch_size = 1000
total_rows = len(df)
jibun_series = []
unprocessed_idx = []
count = 0

for start in range(0, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    print(f"📦 처리 중: {start} ~ {end - 1}행")

    for i in range(start, end):
        try:
            addr = df[address_col].iloc[i]

            if "(" in addr and ")" in addr:
                jibun = handle_parenthesis_address(addr, index=i)
                jibun_series.append(jibun)
            else:
                jibun = convert_to_jibun(addr, index=i)
                if jibun and jibun.strip():
                    jibun_series.append(jibun)
                    count += 1
                else:
                    jibun_series.append(None)
                time.sleep(0.2)

        except Exception as e:
            print(f"❌ 인덱스 {i} 요청 중 오류: {e}")
            jibun_series.append(None)
            unprocessed_idx.append(i)

    print(f"✅ 현재까지 누적 처리: {count}건 / 실패: {len(unprocessed_idx)}건")

print(f"🎉 전체 처리 완료 (성공: {count}건 / 실패: {len(unprocessed_idx)}건)")

# 3. Jibun 열을 dutyAddr 바로 오른쪽에 삽입
insert_at = df.columns.get_loc(address_col) + 1
df.insert(insert_at, "Jibun", jibun_series)

In [162]:
df['Jibun']

0          전라북도 무주군 무주읍
1          전라북도 무주군 무주읍
2          전라북도 임실군 오수면
3          전라북도 임실군 관촌면
4          전라북도 임실군 임실읍
              ...      
77579    전라북도 전주시 삼천동1가
77580      전라북도 완주군 삼례읍
77581    전라북도 전주시 인후동2가
77582    전라북도 전주시 호성동1가
77583    전라북도 전주시 인후동1가
Name: Jibun, Length: 77584, dtype: object

In [ ]:
output_path = "C:/Users/hk522/Desktop/캡스톤 팀프로젝트/병원 풀데이터 지번주소.xlsx"
df.to_excel(output_path, index=False)
print(f"✅ 엑셀 파일로 저장 완료: {output_path}")

✅ 엑셀 파일로 저장 완료: C:/Users/hk522/Desktop/병원 풀데이터 지번주소소.xlsx


In [ ]:
batch_size = 5000
total_rows = len(pharmacy_df)
jibun_series = []
unprocessed_idx = []
count = 0

for start in range(0, total_rows, batch_size):
    end = min(start + batch_size, total_rows)
    print(f"📦 처리 중: {start} ~ {end - 1}행")

    for i in range(start, end):
        try:
            addr = pharmacy_df[address_col].iloc[i]

            if "(" in addr and ")" in addr:
                jibun = handle_parenthesis_address(addr, index=i)
                jibun_series.append(jibun)
            else:
                jibun = convert_to_jibun(addr, index=i)
                if jibun and jibun.strip():
                    jibun_series.append(jibun)
                    count += 1
                else:
                    jibun_series.append(None)
                time.sleep(0.2)

        except Exception as e:
            print(f"❌ 인덱스 {i} 요청 중 오류: {e}")
            jibun_series.append(None)
            unprocessed_idx.append(i)

    print(f"✅ 현재까지 누적 처리: {count}건 / 실패: {len(unprocessed_idx)}건")

print(f"🎉 전체 처리 완료 (성공: {count}건 / 실패: {len(unprocessed_idx)}건)")

# 3. Jibun 열을 dutyAddr 바로 오른쪽에 삽입
insert_at = pharmacy_df.columns.get_loc(address_col) + 1
pharmacy_df.insert(insert_at, "Jibun", jibun_series)

In [ ]:
output_path = "C:/Users/hk522/Desktop/캡스톤 팀프로젝트/약국 풀데이터 지번주소.xlsx"
pharmacy_df.to_excel(output_path, index=False)
print(f"✅ 엑셀 파일로 저장 완료: {output_path}")

✅ 엑셀 파일로 저장 완료: C:/Users/hk522/Desktop/약국 풀데이터 지번주소.xlsx


In [10]:
df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/병원 풀데이터 지번주소.xlsx")
pharmacy_df = pd.read_excel("C:/Users/hk522/Desktop/캡스톤 팀프로젝트/약국 풀데이터 지번주소.xlsx")

In [58]:
emd_cache = {}  # emd → jibun 변환 결과 캐시

def handle_parenthesis_address(addr, index=None):
    try:
        if not addr or not isinstance(addr, str):
            return None

        # 시도 추출
        sido = next((s for s in 시도_목록 if s in addr), None)
        if not sido:
            return None

        # 시군구 추출
        sgg = next((s for s in 시군구_목록 if s in addr and sido in addr), None)
        if not sgg:
            return None

        # emd 추출 (법정동)
        emd = next((candidate for candidate in 법정동_목록 if candidate in addr and sgg in addr and sido in addr), None)
        if not emd:
            return None

        # 캐시 확인
        if emd in emd_cache:
            return emd_cache[emd]

        # 변환은 전체 주소(addr)를 기반으로!
        jibun = convert_to_jibun(addr, index)
        if jibun:
            emd_cache[emd] = jibun
            return jibun
        else:
            return None

    except Exception as e:
        print(f"❌ handle_parenthesis_address(): {index}행 오류 - {e}")
        return None

In [60]:
# ---------- 3-2. 샘플 50건 중 괄호 있는 행은 직접 처리 ----------
# 1. 기존 df에서 앞 50행을 복사하여 새 데이터프레임 생성
sample_size = 50
df_sample = df.iloc[:sample_size].copy()

# 2. 지번주소 결과를 담을 리스트 생성
jibun_series = df_sample["Jibun"].tolist() if "Jibun" in df_sample.columns else [None] * sample_size

# 3. emd 캐시 저장용 딕셔너리
emd_cache = {}

# 4. 주소 변환 및 저장
count = 0

for i in range(sample_size):
    addr = df_sample[address_col].iloc[i]

    try:
        if "(" in addr and ")" in addr:
            jibun = handle_parenthesis_address(addr, index=i)

            if jibun and jibun.strip():
                jibun_series[i] = jibun
                count += 1

        # 괄호 없는 경우는 작업 안 함

    except Exception as e:
        print(f"❌ 인덱스 {i} 오류: {e}")

print(f"✅ 샘플 {sample_size}건 처리 완료 (API 성공: {count}건)")

# 5. "Jibun" 열 덮어쓰기
df_sample["Jibun"] = jibun_series

🔧 [디버그] 7행 - 콤마 이후 제거: '경상남도 김해시 분성로 321, 골든에비뉴 205호 (서상동)' → '경상남도 김해시 분성로 321'
🔧 [디버그] 11행 - 콤마 이후 제거: '경기도 오산시 수청로 189, 601,602호 (금암동, 웅신아트프라자)' → '경기도 오산시 수청로 189'
🔧 [디버그] 12행 - 콤마 이후 제거: '경기도 오산시 내삼미로79번길 11, 2층 207,208호 (수청동, 선우중앙타워)' → '경기도 오산시 내삼미로79번길 11'
🔧 [디버그] 14행 - 콤마 이후 제거: '경기도 오산시 북삼미로 175, 삼성본병원 A동 및 B동 (외삼미동)' → '경기도 오산시 북삼미로 175'
🔧 [디버그] 15행 - 콤마 이후 제거: '전북특별자치도 익산시 선화로 88, 강남빌딩 3층 302호 (모현동1가)' → '전북특별자치도 익산시 선화로 88'
🔧 [디버그] 16행 - 콤마 이후 제거: '충청남도 천안시 동남구 터미널9길 25, 303호 (신부동, 신비프라자)' → '충청남도 천안시 동남구 터미널9길 25'
🔧 [디버그] 18행 - 콤마 이후 제거: '충청남도 천안시 동남구 충절로 278, 1,2,3층 (구성동)' → '충청남도 천안시 동남구 충절로 278'
🔧 [디버그] 20행 - 콤마 이후 제거: '제주특별자치도 제주시 과원북4길 33, 2층 (노형동, 은혜빌딩)' → '제주특별자치도 제주시 과원북4길 33'
🔧 [디버그] 23행 - 콤마 이후 제거: '충청남도 당진시 송악읍 반촌로 70-16, 3층 (송악읍)' → '충청남도 당진시 송악읍 반촌로 70-16'
🔧 [디버그] 26행 - 콤마 이후 제거: '부산광역시 남구 분포로 111, 1005동 305호 (용호동, LG메트로시티 상가 )' → '부산광역시 남구 분포로 111'
🔧 [디버그] 31행 - 콤마 이후 제거: '대전광역시 유성구 테크노중앙로 32, 대덕테크노벨리 4층 (관평동)' → '대전광역시 유성구 테크노중앙로 32'
🔧 [디버그] 32행 - 콤마 이후 제거:

In [61]:
df_sample['Jibun']

0            전라북도 무주군 무주읍
1            전라북도 무주군 무주읍
2            전라북도 임실군 오수면
3            전라북도 임실군 관촌면
4            전라북도 임실군 임실읍
5            전라북도 임실군 오수면
6            경상남도 김해시 무계동
7            경상남도 김해시 서상동
8            전라북도 김제시 요촌동
9            전라북도 김제시 검산동
10           전라북도 김제시 요촌동
11            경기도 오산시 금암동
12            경기도 오산시 수청동
13            경기도 오산시 금암동
14           경기도 오산시 외삼미동
15         전라북도 익산시 모현동1가
16       충청남도 천안시 동남구 신부동
17       충청남도 천안시 동남구 신부동
18       충청남도 천안시 동남구 구성동
19           전라북도 완주군 봉동읍
20            제주도 제주시 노형동
21            제주도 제주시 노형동
22           충청남도 당진시 송악읍
23           충청남도 당진시 송악읍
24       충청남도 천안시 동남구 구성동
25           부산광역시 남구 대연동
26           부산광역시 남구 용호동
27           부산광역시 남구 대연동
28           경상남도 김해시 진영읍
29           부산광역시 남구 용호동
30           부산광역시 남구 대연동
31          대전광역시 유성구 관평동
32          대전광역시 유성구 원내동
33           대전광역시 동구 판암동
34           경상남도 거제시 옥포동
35       충청남도 천안시 동남구 신방동
36       충청북도 청주시 상당구 금천동
37           충청남도 예산군 삽교읍
38          

In [49]:
df['Jibun']

0          전라북도 무주군 무주읍
1          전라북도 무주군 무주읍
2          전라북도 임실군 오수면
3          전라북도 임실군 관촌면
4          전라북도 임실군 임실읍
              ...      
77579    전라북도 전주시 삼천동1가
77580      전라북도 완주군 삼례읍
77581    전라북도 전주시 인후동2가
77582    전라북도 전주시 호성동1가
77583    전라북도 전주시 인후동1가
Name: Jibun, Length: 77584, dtype: object

In [36]:
df['dutyAddr'].iloc[16]

'충청남도 천안시 동남구 터미널9길 25, 303호 (신부동, 신비프라자)'

In [59]:
handle_parenthesis_address('충청남도 천안시 동남구 터미널9길 25, 303호 (신부동, 신비프라자)')

🔧 [디버그] None행 - 콤마 이후 제거: '충청남도 천안시 동남구 터미널9길 25, 303호 (신부동, 신비프라자)' → '충청남도 천안시 동남구 터미널9길 25'


'충청남도 천안시 동남구 신부동'

In [ ]:
total_rows = len(df)
jibun_series = df["Jibun"].tolist() if "Jibun" in df.columns else [None] * total_rows
unprocessed_idx = []
count = 0

print(f"📦 전체 {total_rows}행 처리 시작")

for i in range(total_rows):
    try:
        addr = df[address_col].iloc[i]

        if "(" in addr and ")" in addr:
            jibun = handle_parenthesis_address(addr, index=i)
            if jibun and jibun.strip():
                jibun_series[i] = jibun
                count += 1
        # 괄호 없는 경우는 아무 작업도 하지 않음

    except Exception as e:
        print(f"❌ 인덱스 {i} 요청 중 오류: {e}")
        unprocessed_idx.append(i)

print(f"🎉 전체 처리 완료 (성공: {count}건 / 실패: {len(unprocessed_idx)}건)")

# Jibun 열 덮어쓰기
df["Jibun"] = jibun_series

📦 전체 77584행 처리 시작
🔧 [디버그] 51행 - 콤마 이후 제거: '서울특별시 구로구 구로중앙로 134, 4층 (구로동, 신구로자이나인스에비뉴)' → '서울특별시 구로구 구로중앙로 134'
🔧 [디버그] 52행 - 콤마 이후 제거: '서울특별시 구로구 개봉로 126, 구로다나병원빌딩 (개봉동)' → '서울특별시 구로구 개봉로 126'
🔧 [디버그] 55행 - 콤마 이후 제거: '전북특별자치도 전주시 덕진구 송천중앙로 60, 3~6층 (송천동1가)' → '전북특별자치도 전주시 덕진구 송천중앙로 60'
🔧 [디버그] 56행 - 콤마 이후 제거: '전북특별자치도 전주시 덕진구 정언신로 95, 201호 (인후동1가)' → '전북특별자치도 전주시 덕진구 정언신로 95'
🔧 [디버그] 57행 - 콤마 이후 제거: '전북특별자치도 전주시 덕진구 가련산로 13, 3층 (덕진동2가)' → '전북특별자치도 전주시 덕진구 가련산로 13'
🔧 [디버그] 64행 - 콤마 이후 제거: '부산광역시 남구 우암로 168, 2층 (우암동)' → '부산광역시 남구 우암로 168'
🔧 [디버그] 73행 - 콤마 이후 제거: '부산광역시 수영구 수영로 693, BNK 수영타워 7층 702호 (수영동)' → '부산광역시 수영구 수영로 693'
🔧 [디버그] 77행 - 콤마 이후 제거: '경상북도 포항시 북구 장량중앙로 55, 미소빌딩 3층 (양덕동)' → '경상북도 포항시 북구 장량중앙로 55'
🔧 [디버그] 80행 - 콤마 이후 제거: '경상남도 김해시 율하6로 67, 율하솔로몬빌딩 602,603,604호 (장유동)' → '경상남도 김해시 율하6로 67'
🔧 [디버그] 82행 - 콤마 이후 제거: '서울특별시 구로구 경인로20길 6, 포스시티 105,106호 (오류동)' → '서울특별시 구로구 경인로20길 6'
🔧 [디버그] 84행 - 콤마 이후 제거: '서울특별시 구로구 경인로 661, 신도림1차푸르지오 238,239호 (신도림동)' → '서울특별시 구로구 경인로 661'
🔧 [디버그